# Bedrock Agent with LangGraph

In this notebook, we show how to use Bedrock agent in LangGraph using the BedrockAgentsRunnable construct. Specifically, we will show how to emulate AgentExecutor with LangGraph. We will use the same example from the `bedrock_agent_roc` notebook i.e., we create a mortgage agent with two tools. The first tool will return the asset values of a given asset holder. The second tool will return the interest rate for a given asset holder with a given asset value.

The Graph is a simple structure with two nodes - agent node and tools node.
The agent node is invoked first so this is the START node. If the agent responds with RoC, then the tools node is invoked, the tool results are given to the agent and the agent is invoked again. Once there are no more tool invocations, the Graph reaches an END state. 


### Step 1: Define the tools for the agent

In [16]:
from langchain_core.tools import tool

@tool("AssetDetail::getAssetValue")
def get_asset_value(asset_holder_id: str) -> str:
    """
        Get the asset value for an owner id

        Args:
            asset_holder_id: The asset holder id

        Returns:
            The asset value for the given asset holder
    
    """
    return f"The total asset value for {asset_holder_id} is 100K"

@tool("AssetDetail::getMortgageRate")
def get_mortgage_rate(asset_holder_id: str, asset_value: str) -> str:
    """
        Get the mortgage rate based on asset value

        Args:
            asset_holder_id: The asset holder id
            asset_value: The value of the asset

        Returns:
            The interest rate for the asset holder and asset value
        
    """
    return (
        f"The mortgage rate for {asset_holder_id} "
        f"with asset value of {asset_value} is 8.87%"
    )

tools = [get_asset_value, get_mortgage_rate]
tools

[StructuredTool(name='AssetDetail::getAssetValue', description='Get the asset value for an owner id\n\nArgs:\n    asset_holder_id: The asset holder id\n\nReturns:\n    The asset value for the given asset holder', args_schema=<class 'pydantic.v1.main.AssetDetail::getAssetValueSchema'>, func=<function get_asset_value at 0x10f79c1f0>),
 StructuredTool(name='AssetDetail::getMortgageRate', description='Get the mortgage rate based on asset value\n\nArgs:\n    asset_holder_id: The asset holder id\n    asset_value: The value of the asset\n\nReturns:\n    The interest rate for the asset holder and asset value', args_schema=<class 'pydantic.v1.main.AssetDetail::getMortgageRateSchema'>, func=<function get_mortgage_rate at 0x11e809dc0>)]

### Step 2: Define the foundation model and instructions for the agent

In [17]:
foundational_model = 'anthropic.claude-3-sonnet-20240229-v1:0'
foundational_model

'anthropic.claude-3-sonnet-20240229-v1:0'

In [18]:
instructions="You are an agent who helps with getting the mortgage rate based on the current asset valuation"
instructions

'You are an agent who helps with getting the mortgage rate based on the current asset valuation'

### Step 3: Define the resource role to use with the Bedrock Agent

In [19]:
import boto3
import json
import time
import uuid

def _create_agent_role(
        agent_region,
        foundational_model
) -> str:
    """
    Create agent resource role prior to creation of agent, at this point we do not have agentId, keep it as wildcard

    Args:
        agent_region: AWS region in which is the Agent if available
        foundational_model: The model used for inference in AWS BedrockAgents
    Returns:
       Agent execution role arn
    """
    try:
        account_id = boto3.client('sts').get_caller_identity().get('Account')
        assume_role_policy_document = json.dumps({
            "Version": "2012-10-17",
            "Statement": [
                {
                    "Effect": "Allow",
                    "Principal": {
                        "Service": "bedrock.amazonaws.com"
                    },
                    "Action": "sts:AssumeRole",
                    "Condition": {
                        "ArnLike": {
                            "aws:SourceArn": f"arn:aws:bedrock:{agent_region}:{account_id}:agent/*"
                        }
                    }
                }
            ]
        })
        managed_policy = {
            "Version": "2012-10-17",
            "Statement": [
                {
                    "Sid": "AmazonBedrockAgentBedrockFoundationModelStatement",
                    "Effect": "Allow",
                    "Action": "bedrock:InvokeModel",
                    "Resource": [
                        f"arn:aws:bedrock:{agent_region}::foundation-model/{foundational_model}"
                    ]
                }
            ]
        }
        role_name = f'bedrock_agent_{uuid.uuid4()}'
        iam_client = boto3.client('iam')
        response = iam_client.create_role(
            RoleName=role_name,
            AssumeRolePolicyDocument=assume_role_policy_document,
            Description='Role for Bedrock Agent'
        )
        iam_client.put_role_policy(
            RoleName=role_name,
            PolicyName=f'AmazonBedrockAgentBedrockFoundationModelPolicy_{uuid.uuid4()}',
            PolicyDocument=json.dumps(managed_policy)
        )
        time.sleep(2)
        return response.get('Role', {}).get('Arn', '')

    except Exception as exception:
        raise exception

agent_resource_role_arn = _create_agent_role(
            agent_region='us-west-2',
            foundational_model=foundational_model)

agent_resource_role_arn

'arn:aws:iam::058264503912:role/bedrock_agent_632274c9-e14d-4c84-af81-f57ea73979eb'

### Step 4: Now create the Bedrock Agent

In [20]:
from langchain_aws.agents.base import BedrockAgentsRunnable

agent = BedrockAgentsRunnable.create_agent(
            agent_name="langgraph_interest_rate_agent",
            agent_resource_role_arn=agent_resource_role_arn,
            model=foundational_model,
            instructions="""
            You are an agent who helps with getting the mortgage rate based on the current asset valuation""",
            tools=tools,
        )
agent

BedrockAgentsRunnable(agent_id='T5JY8RSNOJ', client=<botocore.client.AgentsforBedrockRuntime object at 0x10827a5e0>)

### Step 5: Define the State for your Graph

In [34]:
import operator
from typing import TypedDict, Annotated, Tuple
from typing import Union
from langchain_aws.agents.base import BedrockAgentAction, BedrockAgentFinish

class AgentState(TypedDict):
    """
        Defines the state for the Graph
    """
    input: str
    """Input to the agent"""
    output: Union[BedrockAgentAction, BedrockAgentFinish, None]
    """Output from the agent which could be either a BedrockAgentAction or BedrockAgentFinish"""
    intermediate_steps: Annotated[list[tuple[BedrockAgentAction, str]], operator.add]
    """Intermediate steps which are tracked in the state"""

### Step 6: Define the nodes for your LangGraph

In [35]:
from langgraph.prebuilt.tool_executor import ToolExecutor

# the agent node
def run_agent(data):
    agent_outcome = agent.invoke(data)
    return {"output": agent_outcome}

# the tools node
tool_executor = ToolExecutor(tools)

def execute_tools(data):
    # Get the most recent output - this is the key added in the `agent` above
    agent_action = data["output"]
    output = tool_executor.invoke(agent_action[0])
    tuple_output = agent_action[0], output
    print(f"Tuple output is {tuple_output}")
    return {"intermediate_steps": [tuple_output]}

### Step 7: Define the conditional function to invoke the tool node or to reach end if there are no actions

In [36]:
def should_continue(data):
    output_ = data["output"]

    # If the agent outcome is a list of BedrockAgentActions, then we continue to tool execution
    if isinstance(output_, list) and len(output_) > 0 and isinstance(output_[0], BedrockAgentAction):
        return "continue"

    # If the agent outcome is a BedrockAgentAction, then we continue to tool execution
    if isinstance(output_, BedrockAgentAction):
        return "continue"

    # If the agent outcome is an AgentFinish, then we return `exit` string
    # This will be used when setting up the graph to define the flow
    if isinstance(output_, BedrockAgentFinish):
        return "end"

    # Unknown output from the agent, end the graph
    return "end"

### Step 8: Finally, build the graph

In [39]:
from langgraph.graph import END, StateGraph, START

workflow = StateGraph(AgentState)

# Define the two nodes we will cycle between
workflow.add_node("agent", run_agent)
workflow.add_node("action", execute_tools)

# Set the entrypoint as `agent`
# This means that this node is the first one called
workflow.add_edge(START, "agent")

# We now add a conditional edge
workflow.add_conditional_edges(
    # First, we define the start node. We use `agent`.
    # This means these are the edges taken after the `agent` node is called.
    "agent",
    # Next, we pass in the function that will determine which node is called next.
    should_continue,
    # Finally we pass in a mapping.
    # The keys are strings, and the values are other nodes.
    # END is a special node marking that the graph should finish.
    # What will happen is we will call `should_continue`, and then the output of that
    # will be matched against the keys in this mapping.
    # Based on which one it matches, that node will then be called.
    {
        # If `tools`, then we call the tool node.
        "continue": "action",
        # Otherwise we finish.
        "end": END,
    },
)

workflow.add_edge("action", "agent")

### Step 9: Compile the graph

In [40]:
application = workflow.compile(debug=True)

application

CompiledStateGraph(nodes={'__start__': PregelNode(config={'tags': ['langsmith:hidden'], 'metadata': {}, 'configurable': {}}, channels=['__start__'], triggers=['__start__'], writers=[ChannelWrite<input,output,intermediate_steps>(recurse=True, writes=[ChannelWriteEntry(channel='input', value=<object object at 0x10b08c9c0>, skip_none=False, mapper=_get_state_key(recurse=False)), ChannelWriteEntry(channel='output', value=<object object at 0x10b08c9c0>, skip_none=False, mapper=_get_state_key(recurse=False)), ChannelWriteEntry(channel='intermediate_steps', value=<object object at 0x10b08c9c0>, skip_none=False, mapper=_get_state_key(recurse=False))], require_at_least_one_of=['input', 'output', 'intermediate_steps']), ChannelWrite<start:agent>(recurse=True, writes=[ChannelWriteEntry(channel='start:agent', value='__start__', skip_none=False, mapper=None)], require_at_least_one_of=None)]), 'agent': PregelNode(config={'tags': [], 'metadata': {}, 'configurable': {}}, channels={'input': 'input', 'o

### Step 10: Invoke the graph

In [41]:
inputs = {"input": "what is my mortgage rate for id AVC-1234?"}
final_state = application.invoke(inputs)

final_state

[{'step': -2}:checkpoint] State at the end of step {'step': -2}:
{'intermediate_steps': []}
[0:tasks] Starting step 0 with 1 task:
- __start__ -> {'input': 'what is my mortgage rate for id AVC-1234?'}
[0:writes] Finished step 0 with writes to 1 channel:
- input -> 'what is my mortgage rate for id AVC-1234?'
[{'step': -2}:checkpoint] State at the end of step {'step': -2}:
{'input': 'what is my mortgage rate for id AVC-1234?', 'intermediate_steps': []}
[1:tasks] Starting step 1 with 1 task:
- agent -> {'input': 'what is my mortgage rate for id AVC-1234?',
 'intermediate_steps': [],
 'output': None}
[1:writes] Finished step 1 with writes to 1 channel:
- output -> [BedrockAgentAction(tool='AssetDetail::getAssetValue', tool_input={'asset_holder_id': 'AVC-1234'}, log='{"returnControl": {"invocationId": "b2d84cb6-6fa6-4dc1-a0cd-c9f26e9adb2a", "invocationInputs": [{"functionInvocationInput": {"actionGroup": "AssetDetail", "function": "getAssetValue", "parameters": [{"name": "asset_holder_id", 

{'input': 'what is my mortgage rate for id AVC-1234?',
 'output': BedrockAgentFinish(return_values={'output': 'The mortgage rate for the asset holder with id AVC-1234 is 8.87%'}, log='The mortgage rate for the asset holder with id AVC-1234 is 8.87%', session_id='43118b2b-c782-4ebb-9e9b-e1bb31eedbc1'),
 'intermediate_steps': [(BedrockAgentAction(tool='AssetDetail::getAssetValue', tool_input={'asset_holder_id': 'AVC-1234'}, log='{"returnControl": {"invocationId": "b2d84cb6-6fa6-4dc1-a0cd-c9f26e9adb2a", "invocationInputs": [{"functionInvocationInput": {"actionGroup": "AssetDetail", "function": "getAssetValue", "parameters": [{"name": "asset_holder_id", "type": "string", "value": "AVC-1234"}]}}]}}', session_id='43118b2b-c782-4ebb-9e9b-e1bb31eedbc1'),
   'The total asset value for AVC-1234 is 100K'),
  (BedrockAgentAction(tool='AssetDetail::getMortgageRate', tool_input={'asset_value': '100K', 'asset_holder_id': 'AVC-1234'}, log='{"returnControl": {"invocationId": "c10aafca-8901-41c3-91d7-21

### Step 11: Cleanup

In [42]:
def delete_agent_role(agent_resource_role_arn: str):
    """
    Delete agent resource role

    Args:
       agent_resource_role_arn: Associated Agent execution role arn
    """
    try:
        iam_client = boto3.client('iam')
        role_name = agent_resource_role_arn.split('/')[-1]
        inline_policies = iam_client.list_role_policies(RoleName=role_name)
        for inline_policy_name in inline_policies.get('PolicyNames', []):
            iam_client.delete_role_policy(
                RoleName=role_name,
                PolicyName=inline_policy_name
            )
        iam_client.delete_role(
            RoleName=role_name
        )
    except Exception as exception:
        raise exception


def delete_agent(agent_id):
    bedrock_client = boto3.client('bedrock-agent')
    bedrock_client.delete_agent(agentId=agent_id, skipResourceInUseCheck=True)


delete_agent(agent_id=agent.agent_id)
delete_agent_role(agent_resource_role_arn=agent_resource_role_arn)